In [2]:
import os
import cv2
import numpy as np

In [12]:
def preprocess_frame(frame, target_size=(100, 55)):
    """
    Preprocesses a frame for gaze prediction.
    Args:
    - frame: The input image frame (assumed to be in BGR format as per OpenCV standard)
    - target_size: The target size to which the frame should be resized (width, height)

    Returns:
    - Preprocessed frame
    """
    # Check if image is loaded correctly
    if frame is None:
        raise ValueError("Invalid input frame")

    # Convert to grayscale
    gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply a binary threshold to get a binary image
    _, binary_image = cv2.threshold(gray_image, 1, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found in the frame")

    # Find the largest contour based on area
    largest_contour = max(contours, key=cv2.contourArea)

    # Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image using the bounding box
    cropped_image = frame[y:y+h, x:x+w]

    # Resize the cropped image to the target size
    resized_image = cv2.resize(cropped_image, target_size)
    #show image
    cv2.imshow('image',resized_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Convert to float and normalize
    preprocessed_image = resized_image.astype(np.float32) / 255.0

    return preprocessed_image

In [13]:
image = cv2.imread('../MPIIGaze/Data/Original/p00/day01/0001.jpg')
preprocess_frame(image)

array([[[0.00784314, 0.        , 0.        ],
        [0.00784314, 0.        , 0.        ],
        [0.00784314, 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ]],

       [[0.00784314, 0.        , 0.        ],
        [0.00784314, 0.        , 0.        ],
        [0.00784314, 0.        , 0.        ],
        ...,
        [0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.00392157, 0.00392157, 0.00392157]],

       [[0.01176471, 0.        , 0.        ],
        [0.01176471, 0.        , 0.        ],
        [0.0627451 , 0.05098039, 0.05098039],
        ...,
        [0.01960784, 0.01960784, 0.01960784],
        [0.01176471, 0.01176471, 0.01176471],
        [0.01568628, 0.01568628, 0.01568628]],

       ...,

       [[0.00784314, 0.        , 0.        ],
        [0.01568628, 0.00784314, 0.00784314],
        [0.0627451 , 0

In [ ]:
def save_batch(images, gaze_data, batch_number, last_processed_dir):
    np.save(f'images_batch_{batch_number}.npy', images)
    np.save(f'gaze_data_batch_{batch_number}.npy', gaze_data)
    with open('progress_log.txt', 'w') as file:
        file.write(f"{batch_number}\n{last_processed_dir}")
    print(f"Saved batch {batch_number}.")

def load_and_preprocess_images_and_annotations(base_path, batch_size=2000):
    images = []
    gaze_data = []
    batch_number = 0
    last_processed_dir = ""

    # Attempt to resume from the last batch and directory
    try:
        with open('progress_log.txt', 'r') as file:
            batch_number, last_processed_dir = file.read().strip().split('\n')
            batch_number = int(batch_number)
            print(f"Resuming from batch {batch_number} in directory {last_processed_dir}.")
    except FileNotFoundError:
        print("No progress log found. Starting from the beginning.")

    # Traverse the base_path recursively
    for subdir, dirs, files in os.walk(base_path):
        if subdir < last_processed_dir:
            continue

        sorted_files = sorted(files)
        for file in sorted_files:
            if file.endswith('.jpg'):
                image_path = os.path.join(subdir, file)

                # Assuming each image has a corresponding line in annotation.txt
                annotation_file = os.path.join(subdir, 'annotation.txt')
                with open(annotation_file, 'r') as ann_file:
                    annotations = [line.strip().split() for line in ann_file]

                # Match image to its annotation (by index)
                image_index = sorted_files.index(file)
                if image_index < len(annotations):
                    annotation = annotations[image_index]
                else:
                    print(f"No annotation for image: {file}")
                    continue

                # Read and preprocess the image
                image = cv2.imread(image_path)
                preprocessed_image = preprocess_frame(image)

                # Append the processed image and gaze data to the lists
                images.append(preprocessed_image)
                gaze_data.append(annotation)                                                                                                                                                                              

                if len(images) >= batch_size:
                    save_batch(images, gaze_data, batch_number, subdir)
                    images = []
                    gaze_data = []
                    batch_number += 1

    # Save any remaining images and gaze data
    if images:
        save_batch(images, gaze_data, batch_number, subdir)
    
    return images, gaze_data, batch_number